# Language Translation
In this project, you’re going to take a peek into the realm of neural network machine translation.  You’ll be training a sequence to sequence model on a dataset of English and French sentences that can translate new sentences from English to French.
## Get the Data
Since translating the whole language of English to French will take lots of time to train, we have provided you with a small portion of the English corpus.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

## Explore the Data
Play around with view_sentence_range to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 0 to 10:
new jersey is sometimes quiet during autumn , and it is snowy in april .
the united states is usually chilly during july , and it is usually freezing in november .
california is usually quiet during march , and it is usually hot in june .
the united states is sometimes mild during june , and it is cold in september .
your least liked fruit is the grape , but my least liked is the apple .
his favorite fruit is the orange , but my favorite is the grape .
paris is relaxing during december , but it is usually chilly in july .
new jersey is busy during spring , and it is never hot in march .
our least liked fruit is the lemon , but my least liked is the grape .
the united states is sometimes busy during january , and it is sometimes warm in november .

French sentences 0 to 10:
new jersey est parfois calme pendant l' automne 

## Implement Preprocessing Function
### Text to Word Ids
As you did with other RNNs, you must turn the text into a number so the computer can understand it. In the function `text_to_ids()`, you'll turn `source_text` and `target_text` from words to ids.  However, you need to add the `<EOS>` word id at the end of `target_text`.  This will help the neural network predict when the sentence should end.

You can get the `<EOS>` word id by doing:
```python
target_vocab_to_int['<EOS>']
```
You can get other word ids using `source_vocab_to_int` and `target_vocab_to_int`.

In [3]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """
    # TODO: Implement Function
    
    source_sentences = [sentence for sentence in source_text.split('\n')]
    target_sentences = [sentence + ' <EOS>' for sentence in target_text.split('\n')]
    
    source_id_text = [[source_vocab_to_int[word] for word in sentence.split()] for sentence in source_sentences]
    target_id_text = [[target_vocab_to_int[word] for word in sentence.split()] for sentence in target_sentences]
    
    return source_id_text, target_id_text


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_text_to_ids(text_to_ids)

Tests Passed


### Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [4]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np
import helper
import problem_unittests as tests

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

### Check the Version of TensorFlow and Access to GPU
This will check to make sure you have the correct version of TensorFlow and access to a GPU

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
from tensorflow.python.layers.core import Dense

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.1.0


/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:15: UserWarning: No GPU found. Please use a GPU to train your neural network.
  from ipykernel import kernelapp as app


## Build the Neural Network
You'll build the components necessary to build a Sequence-to-Sequence model by implementing the following functions below:
- `model_inputs`
- `process_decoder_input`
- `encoding_layer`
- `decoding_layer_train`
- `decoding_layer_infer`
- `decoding_layer`
- `seq2seq_model`

### Input
Implement the `model_inputs()` function to create TF Placeholders for the Neural Network. It should create the following placeholders:

- Input text placeholder named "input" using the TF Placeholder name parameter with rank 2.
- Targets placeholder with rank 2.
- Learning rate placeholder with rank 0.
- Keep probability placeholder named "keep_prob" using the TF Placeholder name parameter with rank 0.
- Target sequence length placeholder named "target_sequence_length" with rank 1
- Max target sequence length tensor named "max_target_len" getting its value from applying tf.reduce_max on the target_sequence_length placeholder. Rank 0.
- Source sequence length placeholder named "source_sequence_length" with rank 1

Return the placeholders in the following the tuple (input, targets, learning rate, keep probability, target sequence length, max target sequence length, source sequence length)

In [7]:
def model_inputs():
    """
    Create TF Placeholders for input, targets, learning rate, and lengths of source and target sequences.
    :return: Tuple (input, targets, learning rate, keep probability, target sequence length,
    max target sequence length, source sequence length)
    """
    # TODO: Implement Function
    inputs = tf.placeholder(tf.int32, shape=[None,None], name= "input")
    targets = tf.placeholder(tf.int32, shape=[None,None], name= "targets")
    lrate = tf.placeholder(tf.float32, name= "learning_rate")
    keep_prob = tf.placeholder(tf.float32, name= "keep_prob")
    target_seq_lenth = tf.placeholder(tf.int32, shape=[None], name= "target_sequence_length")
    max_target_len = tf.reduce_max(target_seq_lenth, name= 'max_target_len')
    source_seq_length = tf.placeholder(tf.int32, shape=[None], name= "source_sequence_length")
    return inputs, targets, lrate, keep_prob, target_seq_lenth, max_target_len, source_seq_length


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_model_inputs(model_inputs)

Tests Passed


### Process Decoder Input
Implement `process_decoder_input` by removing the last word id from each batch in `target_data` and concat the GO ID to the begining of each batch.

In [8]:
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for encoding
    :param target_data: Target Placehoder
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param batch_size: Batch Size
    :return: Preprocessed target data
    """
    # TODO: Implement Function
    go_id = tf.constant(target_vocab_to_int['<GO>'], shape=(batch_size,1), dtype=tf.int32)
    processed_input = tf.concat([go_id,target_data[:,:-1]],1)
    return processed_input

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_process_encoding_input(process_decoder_input)

Tests Passed


### Encoding
Implement `encoding_layer()` to create a Encoder RNN layer:
 * Embed the encoder input using [`tf.contrib.layers.embed_sequence`](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/embed_sequence)
 * Construct a [stacked](https://github.com/tensorflow/tensorflow/blob/6947f65a374ebf29e74bb71e36fd82760056d82c/tensorflow/docs_src/tutorials/recurrent.md#stacking-multiple-lstms) [`tf.contrib.rnn.LSTMCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/LSTMCell) wrapped in a [`tf.contrib.rnn.DropoutWrapper`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/DropoutWrapper)
 * Pass cell and embedded input to [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)

In [9]:
from imp import reload
reload(tests)

def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   encoding_embedding_size):
    """
    Create encoding layer
    :param rnn_inputs: Inputs for the RNN
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param keep_prob: Dropout keep probability
    :param source_sequence_length: a list of the lengths of each sequence in the batch
    :param source_vocab_size: vocabulary size of source data
    :param encoding_embedding_size: embedding size of source data
    :return: tuple (RNN output, RNN state)
    """
    # TODO: Implement Function
    def build_cell(rnn_size, keep_prob):
        lstm = tf.contrib.rnn.LSTMCell(rnn_size)
        lstm_drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        return lstm_drop
    # Stack them all
    stacked_lstm = tf.contrib.rnn.MultiRNNCell([build_cell(rnn_size, keep_prob) for _ in range(num_layers)])
    # Creathe embedding layer.
    embed_encoder = tf.contrib.layers.embed_sequence(rnn_inputs, vocab_size = source_vocab_size, embed_dim = encoding_embedding_size)
    # If we don't have an initial zero state, provide a dtype.
    output, state = tf.nn.dynamic_rnn(stacked_lstm, embed_encoder, source_sequence_length, dtype=tf.float32)
    return output, state

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_encoding_layer(encoding_layer)

Tests Passed


### Decoding - Training
Create a training decoding layer:
* Create a [`tf.contrib.seq2seq.TrainingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/TrainingHelper) 
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [10]:

def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_summary_length: The length of the longest sequence in the batch
    :param output_layer: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing training logits and sample_id
    """
    # TODO: Implement Function
    trainig_helper = tf.contrib.seq2seq.TrainingHelper(dec_embed_input, target_sequence_length)
    basic_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, trainig_helper, encoder_state, output_layer)
    f_output, _ = tf.contrib.seq2seq.dynamic_decode(basic_decoder,maximum_iterations=max_summary_length)
    return f_output

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_train(decoding_layer_train)

Tests Passed


### Decoding - Inference
Create inference decoder:
* Create a [`tf.contrib.seq2seq.GreedyEmbeddingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/GreedyEmbeddingHelper)
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [12]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a decoding layer for inference
    :param encoder_state: Encoder state
    :param dec_cell: Decoder RNN Cell
    :param dec_embeddings: Decoder embeddings
    :param start_of_sequence_id: GO ID
    :param end_of_sequence_id: EOS Id
    :param max_target_sequence_length: Maximum length of target sequences
    :param vocab_size: Size of decoder/target vocabulary
    :param decoding_scope: TenorFlow Variable Scope for decoding
    :param output_layer: Function to apply the output layer
    :param batch_size: Batch size
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    # TODO: Implement Function
    start_ids = tf.tile([start_of_sequence_id], [batch_size])
    embedding_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(
        dec_embeddings, start_ids, end_of_sequence_id)
    basic_decoder = tf.contrib.seq2seq.BasicDecoder(
        dec_cell, embedding_helper, encoder_state, output_layer)
    f_output, _= tf.contrib.seq2seq.dynamic_decode(
        basic_decoder,maximum_iterations=max_target_sequence_length)
    return f_output

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_infer(decoding_layer_infer)

Tests Passed


### Build the Decoding Layer
Implement `decoding_layer()` to create a Decoder RNN layer.

* Embed the target sequences
* Construct the decoder LSTM cell (just like you constructed the encoder cell above)
* Create an output layer to map the outputs of the decoder to the elements of our vocabulary
* Use the your `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, target_sequence_length, max_target_sequence_length, output_layer, keep_prob)` function to get the training logits.
* Use your `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, max_target_sequence_length, vocab_size, output_layer, batch_size, keep_prob)` function to get the inference logits.

Note: You'll need to use [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope) to share variables between training and inference.

In [13]:
def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size):
    """
    Create decoding layer
    :param dec_input: Decoder input
    :param encoder_state: Encoder state
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_target_sequence_length: Maximum length of target sequences
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param target_vocab_size: Size of target vocabulary
    :param batch_size: The size of the batch
    :param keep_prob: Dropout keep probability
    :param decoding_embedding_size: Decoding embedding size
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    def build_cell(rnn_size, keep_prob):
        lstm = tf.contrib.rnn.LSTMCell(rnn_size)
        lstm_drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        return lstm_drop

    stacked_lstm = tf.contrib.rnn.MultiRNNCell([build_cell(rnn_size, keep_prob) for _ in range(num_layers)])
    
    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)

    dense_layer = Dense(target_vocab_size,
                         kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
    
    with tf.variable_scope("decode") as scope:
        tr_decoder_output = decoding_layer_train(
            encoder_state, stacked_lstm, dec_embed_input, 
            target_sequence_length, max_target_sequence_length, 
            dense_layer, keep_prob)
        scope.reuse_variables()
        inf_decoder_output = decoding_layer_infer(
            encoder_state, stacked_lstm, dec_embeddings, 
            target_vocab_to_int['<GO>'], target_vocab_to_int['<EOS>'], 
            max_target_sequence_length, target_vocab_size, 
            dense_layer, batch_size, keep_prob)
    
    return tr_decoder_output, inf_decoder_output



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer(decoding_layer)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:

- Encode the input using your `encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob,  source_sequence_length, source_vocab_size, encoding_embedding_size)`.
- Process target data using your `process_decoder_input(target_data, target_vocab_to_int, batch_size)` function.
- Decode the encoded input using your `decoding_layer(dec_input, enc_state, target_sequence_length, max_target_sentence_length, rnn_size, num_layers, target_vocab_to_int, target_vocab_size, batch_size, keep_prob, dec_embedding_size)` function.

In [14]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  source_sequence_length, target_sequence_length,
                  max_target_sentence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence part of the neural network
    :param input_data: Input placeholder
    :param target_data: Target placeholder
    :param keep_prob: Dropout keep probability placeholder
    :param batch_size: Batch Size
    :param source_sequence_length: Sequence Lengths of source sequences in the batch
    :param target_sequence_length: Sequence Lengths of target sequences in the batch
    :param source_vocab_size: Source vocabulary size
    :param target_vocab_size: Target vocabulary size
    :param enc_embedding_size: Decoder embedding size
    :param dec_embedding_size: Encoder embedding size
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    output, state = encoding_layer(input_data, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   enc_embedding_size)
    
    processed_input = process_decoder_input(target_data, target_vocab_to_int, batch_size)
    
    tr_decoder_output, inf_decoder_output = decoding_layer(processed_input, state,
                   target_sequence_length, max_target_sentence_length,
                   rnn_size, num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, dec_embedding_size)
    
    return tr_decoder_output, inf_decoder_output


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_seq2seq_model(seq2seq_model)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `num_layers` to the number of layers.
- Set `encoding_embedding_size` to the size of the embedding for the encoder.
- Set `decoding_embedding_size` to the size of the embedding for the decoder.
- Set `learning_rate` to the learning rate.
- Set `keep_probability` to the Dropout keep probability
- Set `display_step` to state how many steps between each debug output statement

In [16]:
# Number of Epochs
epochs = 10
# Batch Size
batch_size = 512
# RNN Size
rnn_size = 150
# Number of Layers
num_layers = 3
# Embedding Size
encoding_embedding_size = 126
decoding_embedding_size = 126
# Learning Rate
learning_rate = 0.01
# Dropout Keep Probability
keep_probability = 0.5
display_step = True

### Build the Graph
Build the graph using the neural network you implemented.

In [17]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
max_target_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob, target_sequence_length, max_target_sequence_length, source_sequence_length = model_inputs()

    #sequence_length = tf.placeholder_with_default(max_target_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)

    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   source_sequence_length,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)


    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)


Batch and pad the source and target sequences

In [18]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]


def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths


### Train
Train the neural network on the preprocessed data. If you have a hard time getting a good loss, check the forms to see if anyone is having the same problem.

In [ ]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

# Split data to training and validation sets
train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]
valid_source = source_int_text[:batch_size]
valid_target = target_int_text[:batch_size]
(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = next(get_batches(valid_source,
                                                                                                             valid_target,
                                                                                                             batch_size,
                                                                                                             source_vocab_to_int['<PAD>'],
                                                                                                             target_vocab_to_int['<PAD>']))                                                                                                  
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                get_batches(train_source, train_target, batch_size,
                            source_vocab_to_int['<PAD>'],
                            target_vocab_to_int['<PAD>'])):

            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 source_sequence_length: sources_lengths,
                 keep_prob: keep_probability})


            if batch_i % display_step == 0 and batch_i > 0:


                batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     source_sequence_length: sources_lengths,
                     target_sequence_length: targets_lengths,
                     keep_prob: 1.0})


                batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     source_sequence_length: valid_sources_lengths,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 1.0})

                train_acc = get_accuracy(target_batch, batch_train_logits)

                valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)

                print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.4f}, Validation Accuracy: {:>6.4f}, Loss: {:>6.4f}'
                      .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Epoch   0 Batch    1/269 - Train Accuracy: 0.2679, Validation Accuracy: 0.3414, Loss: 4.8654
Epoch   0 Batch    2/269 - Train Accuracy: 0.0427, Validation Accuracy: 0.0864, Loss: 4.1494
Epoch   0 Batch    3/269 - Train Accuracy: 0.2788, Validation Accuracy: 0.3417, Loss: 4.4845
Epoch   0 Batch    4/269 - Train Accuracy: 0.2683, Validation Accuracy: 0.3410, Loss: 3.9270
Epoch   0 Batch    5/269 - Train Accuracy: 0.2702, Validation Accuracy: 0.3428, Loss: 3.8507
Epoch   0 Batch    6/269 - Train Accuracy: 0.3139, Validation Accuracy: 0.3428, Loss: 3.5595
Epoch   0 Batch    7/269 - Train Accuracy: 0.3182, Validation Accuracy: 0.3487, Loss: 3.4791
Epoch   0 Batch    8/269 - Train Accuracy: 0.2832, Validation Accuracy: 0.3478, Loss: 3.5565
Epoch   0 Batch    9/269 - Train Accuracy: 0.3017, Validation Accuracy: 0.3435, Loss: 3.3932
Epoch   0 Batch   10/269 - Train Accuracy: 0.3156, Validation Accuracy: 0.3822, Loss: 3.4075
Epoch   0 Batch   11/269 - Train Accuracy: 0.3411, Validation Accuracy

Epoch   0 Batch   90/269 - Train Accuracy: 0.4621, Validation Accuracy: 0.5161, Loss: 2.0052
Epoch   0 Batch   91/269 - Train Accuracy: 0.4776, Validation Accuracy: 0.5012, Loss: 1.8659
Epoch   0 Batch   92/269 - Train Accuracy: 0.5021, Validation Accuracy: 0.5193, Loss: 1.8406
Epoch   0 Batch   93/269 - Train Accuracy: 0.5160, Validation Accuracy: 0.5143, Loss: 1.7736
Epoch   0 Batch   94/269 - Train Accuracy: 0.5065, Validation Accuracy: 0.5289, Loss: 1.8234
Epoch   0 Batch   95/269 - Train Accuracy: 0.4847, Validation Accuracy: 0.5048, Loss: 1.8099
Epoch   0 Batch   96/269 - Train Accuracy: 0.4793, Validation Accuracy: 0.5057, Loss: 1.8661
Epoch   0 Batch   97/269 - Train Accuracy: 0.5071, Validation Accuracy: 0.5228, Loss: 1.9697
Epoch   0 Batch   98/269 - Train Accuracy: 0.4756, Validation Accuracy: 0.4808, Loss: 1.7617
Epoch   0 Batch   99/269 - Train Accuracy: 0.4965, Validation Accuracy: 0.5423, Loss: 2.0209
Epoch   0 Batch  100/269 - Train Accuracy: 0.4817, Validation Accuracy

Epoch   0 Batch  179/269 - Train Accuracy: 0.5684, Validation Accuracy: 0.5708, Loss: 0.8640
Epoch   0 Batch  180/269 - Train Accuracy: 0.5757, Validation Accuracy: 0.5768, Loss: 0.8605
Epoch   0 Batch  181/269 - Train Accuracy: 0.5557, Validation Accuracy: 0.5791, Loss: 0.8741
Epoch   0 Batch  182/269 - Train Accuracy: 0.5777, Validation Accuracy: 0.5795, Loss: 0.8573
Epoch   0 Batch  183/269 - Train Accuracy: 0.6197, Validation Accuracy: 0.5664, Loss: 0.7438
Epoch   0 Batch  184/269 - Train Accuracy: 0.5434, Validation Accuracy: 0.5588, Loss: 0.8865
Epoch   0 Batch  185/269 - Train Accuracy: 0.5696, Validation Accuracy: 0.5835, Loss: 0.8411
Epoch   0 Batch  186/269 - Train Accuracy: 0.5509, Validation Accuracy: 0.5898, Loss: 0.8719
Epoch   0 Batch  187/269 - Train Accuracy: 0.5813, Validation Accuracy: 0.5885, Loss: 0.8259
Epoch   0 Batch  188/269 - Train Accuracy: 0.5844, Validation Accuracy: 0.5760, Loss: 0.8067
Epoch   0 Batch  189/269 - Train Accuracy: 0.5775, Validation Accuracy

Epoch   1 Batch    1/269 - Train Accuracy: 0.6307, Validation Accuracy: 0.6520, Loss: 0.6846
Epoch   1 Batch    2/269 - Train Accuracy: 0.6306, Validation Accuracy: 0.6491, Loss: 0.6882
Epoch   1 Batch    3/269 - Train Accuracy: 0.6424, Validation Accuracy: 0.6556, Loss: 0.6910
Epoch   1 Batch    4/269 - Train Accuracy: 0.6195, Validation Accuracy: 0.6493, Loss: 0.6990
Epoch   1 Batch    5/269 - Train Accuracy: 0.6184, Validation Accuracy: 0.6474, Loss: 0.6977
Epoch   1 Batch    6/269 - Train Accuracy: 0.6618, Validation Accuracy: 0.6522, Loss: 0.6471
Epoch   1 Batch    7/269 - Train Accuracy: 0.6282, Validation Accuracy: 0.6460, Loss: 0.6628
Epoch   1 Batch    8/269 - Train Accuracy: 0.6103, Validation Accuracy: 0.6318, Loss: 0.6991
Epoch   1 Batch    9/269 - Train Accuracy: 0.6149, Validation Accuracy: 0.6281, Loss: 0.6895
Epoch   1 Batch   10/269 - Train Accuracy: 0.6087, Validation Accuracy: 0.6320, Loss: 0.6889
Epoch   1 Batch   11/269 - Train Accuracy: 0.6278, Validation Accuracy

Epoch   1 Batch   90/269 - Train Accuracy: 0.6322, Validation Accuracy: 0.6710, Loss: 0.6135
Epoch   1 Batch   91/269 - Train Accuracy: 0.6563, Validation Accuracy: 0.6590, Loss: 0.5659
Epoch   1 Batch   92/269 - Train Accuracy: 0.6578, Validation Accuracy: 0.6578, Loss: 0.5753
Epoch   1 Batch   93/269 - Train Accuracy: 0.6722, Validation Accuracy: 0.6583, Loss: 0.5549
Epoch   1 Batch   94/269 - Train Accuracy: 0.6553, Validation Accuracy: 0.6622, Loss: 0.5891
Epoch   1 Batch   95/269 - Train Accuracy: 0.6635, Validation Accuracy: 0.6687, Loss: 0.5776
Epoch   1 Batch   96/269 - Train Accuracy: 0.6514, Validation Accuracy: 0.6681, Loss: 0.5773
Epoch   1 Batch   97/269 - Train Accuracy: 0.6585, Validation Accuracy: 0.6665, Loss: 0.5690
Epoch   1 Batch   98/269 - Train Accuracy: 0.6541, Validation Accuracy: 0.6610, Loss: 0.5732
Epoch   1 Batch   99/269 - Train Accuracy: 0.6352, Validation Accuracy: 0.6512, Loss: 0.5933
Epoch   1 Batch  100/269 - Train Accuracy: 0.6748, Validation Accuracy

...

Epoch   8 Batch   98/269 - Train Accuracy: 0.8974, Validation Accuracy: 0.9032, Loss: 0.1330
Epoch   8 Batch   99/269 - Train Accuracy: 0.8903, Validation Accuracy: 0.9056, Loss: 0.1228
Epoch   8 Batch  100/269 - Train Accuracy: 0.9097, Validation Accuracy: 0.9013, Loss: 0.1252
Epoch   8 Batch  101/269 - Train Accuracy: 0.8807, Validation Accuracy: 0.9014, Loss: 0.1424
Epoch   8 Batch  102/269 - Train Accuracy: 0.8907, Validation Accuracy: 0.9055, Loss: 0.1216
Epoch   8 Batch  103/269 - Train Accuracy: 0.8903, Validation Accuracy: 0.9086, Loss: 0.1407
Epoch   8 Batch  104/269 - Train Accuracy: 0.8962, Validation Accuracy: 0.9055, Loss: 0.1225
Epoch   8 Batch  105/269 - Train Accuracy: 0.8876, Validation Accuracy: 0.9036, Loss: 0.1320
Epoch   8 Batch  106/269 - Train Accuracy: 0.9098, Validation Accuracy: 0.9057, Loss: 0.1164
Epoch   8 Batch  107/269 - Train Accuracy: 0.8933, Validation Accuracy: 0.9004, Loss: 0.1325
Epoch   8 Batch  108/269 - Train Accuracy: 0.9014, Validation Accuracy

Epoch   8 Batch  187/269 - Train Accuracy: 0.8892, Validation Accuracy: 0.9091, Loss: 0.1187
Epoch   8 Batch  188/269 - Train Accuracy: 0.9028, Validation Accuracy: 0.9092, Loss: 0.1038
Epoch   8 Batch  189/269 - Train Accuracy: 0.9061, Validation Accuracy: 0.9060, Loss: 0.1175
Epoch   8 Batch  190/269 - Train Accuracy: 0.9011, Validation Accuracy: 0.9057, Loss: 0.1114
Epoch   8 Batch  191/269 - Train Accuracy: 0.9027, Validation Accuracy: 0.9024, Loss: 0.1074
Epoch   8 Batch  192/269 - Train Accuracy: 0.9023, Validation Accuracy: 0.9023, Loss: 0.1212
Epoch   8 Batch  193/269 - Train Accuracy: 0.9071, Validation Accuracy: 0.9057, Loss: 0.1188
Epoch   8 Batch  194/269 - Train Accuracy: 0.8956, Validation Accuracy: 0.9032, Loss: 0.1208
Epoch   8 Batch  195/269 - Train Accuracy: 0.8930, Validation Accuracy: 0.9026, Loss: 0.1120
Epoch   8 Batch  196/269 - Train Accuracy: 0.8769, Validation Accuracy: 0.9026, Loss: 0.1098
Epoch   8 Batch  197/269 - Train Accuracy: 0.8923, Validation Accuracy

Epoch   9 Batch    9/269 - Train Accuracy: 0.8935, Validation Accuracy: 0.9064, Loss: 0.1153
Epoch   9 Batch   10/269 - Train Accuracy: 0.9116, Validation Accuracy: 0.9076, Loss: 0.1020
Epoch   9 Batch   11/269 - Train Accuracy: 0.9078, Validation Accuracy: 0.9061, Loss: 0.1167
Epoch   9 Batch   12/269 - Train Accuracy: 0.8842, Validation Accuracy: 0.9063, Loss: 0.1361
Epoch   9 Batch   13/269 - Train Accuracy: 0.9053, Validation Accuracy: 0.9121, Loss: 0.1087
Epoch   9 Batch   14/269 - Train Accuracy: 0.9018, Validation Accuracy: 0.9078, Loss: 0.1188
Epoch   9 Batch   15/269 - Train Accuracy: 0.9033, Validation Accuracy: 0.9099, Loss: 0.1012
Epoch   9 Batch   16/269 - Train Accuracy: 0.9051, Validation Accuracy: 0.9116, Loss: 0.1198
Epoch   9 Batch   17/269 - Train Accuracy: 0.9027, Validation Accuracy: 0.9129, Loss: 0.1019
Epoch   9 Batch   18/269 - Train Accuracy: 0.9007, Validation Accuracy: 0.9097, Loss: 0.1212
Epoch   9 Batch   19/269 - Train Accuracy: 0.9165, Validation Accuracy

Epoch   9 Batch   98/269 - Train Accuracy: 0.9023, Validation Accuracy: 0.9094, Loss: 0.1162
Epoch   9 Batch   99/269 - Train Accuracy: 0.8936, Validation Accuracy: 0.9060, Loss: 0.1122
Epoch   9 Batch  100/269 - Train Accuracy: 0.9082, Validation Accuracy: 0.9068, Loss: 0.1101
Epoch   9 Batch  101/269 - Train Accuracy: 0.8924, Validation Accuracy: 0.9086, Loss: 0.1273
Epoch   9 Batch  102/269 - Train Accuracy: 0.8997, Validation Accuracy: 0.9100, Loss: 0.1168
Epoch   9 Batch  103/269 - Train Accuracy: 0.9055, Validation Accuracy: 0.9111, Loss: 0.1248
Epoch   9 Batch  104/269 - Train Accuracy: 0.8993, Validation Accuracy: 0.9113, Loss: 0.1079
Epoch   9 Batch  105/269 - Train Accuracy: 0.9062, Validation Accuracy: 0.9079, Loss: 0.1160
Epoch   9 Batch  106/269 - Train Accuracy: 0.9075, Validation Accuracy: 0.9107, Loss: 0.1051
Epoch   9 Batch  107/269 - Train Accuracy: 0.8980, Validation Accuracy: 0.9097, Loss: 0.1241
Epoch   9 Batch  108/269 - Train Accuracy: 0.9055, Validation Accuracy

Epoch   9 Batch  187/269 - Train Accuracy: 0.8949, Validation Accuracy: 0.9095, Loss: 0.1090
Epoch   9 Batch  188/269 - Train Accuracy: 0.9059, Validation Accuracy: 0.9128, Loss: 0.1083
Epoch   9 Batch  189/269 - Train Accuracy: 0.9000, Validation Accuracy: 0.9152, Loss: 0.1087
Epoch   9 Batch  190/269 - Train Accuracy: 0.8977, Validation Accuracy: 0.9170, Loss: 0.0983
Epoch   9 Batch  191/269 - Train Accuracy: 0.9032, Validation Accuracy: 0.9168, Loss: 0.1043
Epoch   9 Batch  192/269 - Train Accuracy: 0.9042, Validation Accuracy: 0.9173, Loss: 0.1131
Epoch   9 Batch  193/269 - Train Accuracy: 0.9069, Validation Accuracy: 0.9157, Loss: 0.1112
Epoch   9 Batch  194/269 - Train Accuracy: 0.9026, Validation Accuracy: 0.9146, Loss: 0.1134
Epoch   9 Batch  195/269 - Train Accuracy: 0.9013, Validation Accuracy: 0.9137, Loss: 0.1042
Epoch   9 Batch  196/269 - Train Accuracy: 0.8797, Validation Accuracy: 0.9144, Loss: 0.1099
Epoch   9 Batch  197/269 - Train Accuracy: 0.8952, Validation Accuracy

### Save Parameters
Save the `batch_size` and `save_path` parameters for inference.

In [20]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params(save_path)

# Checkpoint

In [21]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()

## Sentence to Sequence
To feed a sentence into the model for translation, you first need to preprocess it.  Implement the function `sentence_to_seq()` to preprocess new sentences.

- Convert the sentence to lowercase
- Convert words into ids using `vocab_to_int`
 - Convert words not in the vocabulary, to the `<UNK>` word id.

In [22]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    # TODO: Implement Function
    word_ids = []
    for word in sentence.lower().split():
        if word in vocab_to_int:
            word_ids.append(vocab_to_int[word])
        else:
            word_ids.append(vocab_to_int['<UNK>'])
    return word_ids


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_sentence_to_seq(sentence_to_seq)

Tests Passed


## Translate
This will translate `translate_sentence` from English to French.

In [25]:
translate_sentence = 'he saw a old yellow truck .'


"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence]*batch_size,
                                         target_sequence_length: [len(translate_sentence)*2]*batch_size,
                                         source_sequence_length: [len(translate_sentence)]*batch_size,
                                         keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in translate_logits]))
print('  French Words: {}'.format(" ".join([target_int_to_vocab[i] for i in translate_logits])))


INFO:tensorflow:Restoring parameters from checkpoints/dev
Input
  Word Ids:      [83, 185, 84, 64, 24, 173, 201]
  English Words: ['he', 'saw', 'a', 'old', 'yellow', 'truck', '.']

Prediction
  Word Ids:      [21, 231, 347, 97, 122, 297, 181, 88, 1]
  French Words: il a vu une voiture blanche rouillée . <EOS>


## Imperfect Translation
You might notice that some sentences translate better than others.  Since the dataset you're using only has a vocabulary of 227 English words of the thousands that you use, you're only going to see good results using these words.  For this project, you don't need a perfect translation. However, if you want to create a better translation model, you'll need better data.

You can train on the [WMT10 French-English corpus](http://www.statmt.org/wmt10/training-giga-fren.tar).  This dataset has more vocabulary and richer in topics discussed.  However, this will take you days to train, so make sure you've a GPU and the neural network is performing well on dataset we provided.  Just make sure you play with the WMT10 corpus after you've submitted this project.
## Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_language_translation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.